In [1]:
import pandas as pd#read in the dataset
df = pd.read_csv('/content/diabetes.csv')#take a look at the data
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
#check dataset size
df.shape

(768, 9)

In [3]:
#split data into inputs and targets
X = df.drop(columns = ['Outcome'])
y = df['Outcome']

In [4]:
#split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [5]:
import numpy as np
from sklearn.model_selection import GridSearchCV

#create new a knn model
from sklearn.neighbors import KNeighborsClassifier

#create a dictionary of all values we want to test for n_neighbors
knn = KNeighborsClassifier()

#use gridsearch to test all values for n_neighbors
params_knn = {'n_neighbors': np.arange(1, 25)}

#fit model to training data
knn_gs = GridSearchCV(knn, params_knn, cv=5)
knn_gs.fit(X_train, y_train)
print(knn_gs.best_params_)

{'n_neighbors': 8}


In [6]:
#save best model
knn_best = knn_gs.best_estimator_
#check best n_neigbors value
print(knn_gs.best_params_)

{'n_neighbors': 8}


In [7]:
from sklearn.linear_model import LogisticRegression
#create a new logistic regression model
log_reg = LogisticRegression()

#fit the model to the training data
log_reg.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [8]:
#test the three models with the test data and print their accuracy scores
print('knn: {}'.format(knn_best.score(X_test, y_test)))
print('log_reg: {}'.format(log_reg.score(X_test, y_test)))


knn: 0.7359307359307359
log_reg: 0.7575757575757576


In [11]:
from sklearn.ensemble import VotingClassifier
#create a dictionary of our models
base_learners=[('knn', knn_best), ('log_reg', log_reg)]
#create our voting classifier, inputting our models
ensemble_learner = VotingClassifier(base_learners, voting='hard')

In [12]:
#fit model to training data
ensemble_learner.fit(X_train, y_train)
#test our model on the test data
ensemble_learner.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7445887445887446

In [14]:
from sklearn.ensemble import VotingClassifier
#create a dictionary of our models
base_learners=[('knn', knn_best), ('log_reg', log_reg)]
#create our voting classifier, inputting our models
ensemble_learner = VotingClassifier(base_learners, voting='soft')
#fit model to training data
ensemble_learner.fit(X_train, y_train)
#test our model on the test data
ensemble_learner.score(X_test, y_test)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7402597402597403

In [15]:
#create a new random forest classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

#create a dictionary of all values we want to test for n_estimators
params_rf = {'n_estimators': [50, 100, 200]}

#use gridsearch to test all values for n_estimators
rf_gs = GridSearchCV(rf, params_rf, cv=5)

#fit model to training data
rf_gs.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [50, 100, 200]})

In [16]:
#save best model
rf_best = rf_gs.best_estimator_
#check best n_estimators value
print(rf_gs.best_params_)

{'n_estimators': 100}


In [17]:
print('rf: {}'.format(rf_best.score(X_test, y_test)))

rf: 0.7619047619047619


In [18]:
from sklearn.ensemble import AdaBoostClassifier
AdaBoost = AdaBoostClassifier()

#create a dictionary of all values we want to test for n_estimators
params_AdaBoost = {'n_estimators': [50, 100, 200]}

#use gridsearch to test all values for n_estimators
AdaBoost_gs = GridSearchCV(AdaBoost, params_AdaBoost, cv=5)

#fit model to training data
AdaBoost_gs.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=AdaBoostClassifier(),
             param_grid={'n_estimators': [50, 100, 200]})

In [19]:
#save best model
AdaBoost_best = AdaBoost_gs.best_estimator_
#check best n_estimators value
print(AdaBoost_gs.best_params_)

{'n_estimators': 50}


In [20]:
print('AdaBoost: {}'.format(AdaBoost_best.score(X_test, y_test)))

AdaBoost: 0.7662337662337663


In [21]:
from sklearn.ensemble import GradientBoostingClassifier
GMB = GradientBoostingClassifier()

#create a dictionary of all values we want to test for n_estimators
params_GMB = {'learning_rate': [0.01, 0.1, 1.0]}

#use gridsearch to test all values for n_estimators
GMB_gs = GridSearchCV(GMB, params_GMB, cv=5)

#fit model to training data
GMB_gs.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.01, 0.1, 1.0]})

In [22]:
#save best model
GMB_best = AdaBoost_gs.best_estimator_
#check best n_estimators value
print(GMB_gs.best_params_)

{'learning_rate': 0.1}


In [23]:
print('GMB: {}'.format(GMB_best.score(X_test, y_test)))

GMB: 0.7662337662337663


In [25]:
import xgboost as xgb
XGBoost = xgb.XGBClassifier()

#create a dictionary of all values we want to test for n_estimators
params_XGBoost = {'learning_rate': [0.01, 0.1, 1.0]}

#use gridsearch to test all values for learning rate
XGBoost_gs = GridSearchCV(XGBoost , params_XGBoost, cv=5)

#fit model to training data
XGBoost_gs.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.1, 1.0]})

In [26]:
#save best model
XGBoost_best = XGBoost_gs.best_estimator_
#check best n_estimators value
print(XGBoost_gs.best_params_)

{'learning_rate': 0.01}


In [27]:
print('XGBoost: {}'.format(XGBoost_best.score(X_test, y_test)))

XGBoost: 0.7575757575757576


In [36]:
import lightgbm as lgb
LGB = lgb.LGBMClassifier()

#create a dictionary of all values we want to test for n_estimators
params_LGB = {'learning_rate': [0.001, 0.01, 0.1, 1.0],
    'num_leaves': [31, 127],
    'reg_alpha': [0.1, 0.5],
    'min_data_in_leaf': [30, 50, 100, 300, 400]}

#use gridsearch to test all values for learning rate
LGB_gs = GridSearchCV(LGB , params_LGB, cv=5)

#fit model to training data
LGB_gs.fit(X_train, y_train)

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

GridSearchCV(cv=5, estimator=LGBMClassifier(),
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 1.0],
                         'min_data_in_leaf': [30, 50, 100, 300, 400],
                         'num_leaves': [31, 127], 'reg_alpha': [0.1, 0.5]})

In [38]:
#save best model
LGB_best = LGB_gs.best_estimator_
#check best n_estimators value
print(LGB_gs.best_params_)

{'learning_rate': 0.01, 'min_data_in_leaf': 30, 'num_leaves': 31, 'reg_alpha': 0.5}


In [39]:
print('LGB: {}'.format(LGB_best.score(X_test, y_test)))

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
LGB: 0.7532467532467533


In [40]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.1 MB/s eta 0:00:00


In [42]:
from catboost import CatBoostClassifier
CatBoost=CatBoostClassifier()

#create a dictionary of all values we want to test for n_estimators
params_CatBoost = {'depth'         : [3,5,10],
                 'learning_rate' : [0.01,0.1,1],
                  'iterations'    : [5,10,50,100]}

#use gridsearch to test all values for learning rate
CatBoost_gs = GridSearchCV(CatBoost, params_CatBoost, cv=5)

#fit model to training data
CatBoost_gs.fit(X_train, y_train)

Streaming output truncated to the last 5000 lines.
0:	learn: 0.6902974	total: 1.2ms	remaining: 4.81ms
1:	learn: 0.6875250	total: 4.34ms	remaining: 6.51ms
2:	learn: 0.6847215	total: 6.38ms	remaining: 4.25ms
3:	learn: 0.6824649	total: 8.31ms	remaining: 2.08ms
4:	learn: 0.6799294	total: 10.4ms	remaining: 0us
0:	learn: 0.6903969	total: 845us	remaining: 3.38ms
1:	learn: 0.6881085	total: 2.58ms	remaining: 3.86ms
2:	learn: 0.6854638	total: 4.08ms	remaining: 2.72ms
3:	learn: 0.6831414	total: 5.58ms	remaining: 1.39ms
4:	learn: 0.6807343	total: 12.4ms	remaining: 0us
0:	learn: 0.6898841	total: 1.99ms	remaining: 7.97ms
1:	learn: 0.6873874	total: 4.95ms	remaining: 7.42ms
2:	learn: 0.6846961	total: 7.69ms	remaining: 5.13ms
3:	learn: 0.6823034	total: 10.5ms	remaining: 2.62ms
4:	learn: 0.6797217	total: 13.7ms	remaining: 0us
0:	learn: 0.6902740	total: 1.33ms	remaining: 5.34ms
1:	learn: 0.6878362	total: 2.63ms	remaining: 3.94ms
2:	learn: 0.6849668	total: 3.73ms	remaining: 2.48ms
3:	learn: 0.6827127	tota

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x79046c27b7c0>,
             param_grid={'depth': [3, 5, 10], 'iterations': [5, 10, 50, 100],
                         'learning_rate': [0.01, 0.1, 1]})

In [44]:
#save best model
CatBoost_best = CatBoost_gs.best_estimator_
#check best n_estimators value
print(CatBoost_gs.best_params_)

{'depth': 3, 'iterations': 50, 'learning_rate': 0.1}


In [45]:
print('CatBoost: {}'.format(CatBoost_best.score(X_test, y_test)))

CatBoost: 0.7489177489177489


In [58]:
from sklearn.ensemble import StackingClassifier
#models = [('lr',LogisticRegression()),('svm',SVC())
base_learners=[('knn', knn_best), ('log_reg', log_reg), ('rf',rf)]
stacking_ensemble = StackingClassifier(estimators=base_learners)

In [57]:
#fit model to training data
stacking_ensemble.fit(X_train, y_train)
#test our model on the test data
stacking_ensemble.score(X_test, y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

0.7532467532467533